<center> <h1> GPU Filter Banks for Audio </h1> </center>

Names

Cool picture

# Background
![](img/trends1.png)

For audio signal processing problems such as compression, denoising, and various types of pattern recognition, deep learning has taken over. For example, the search interest for "wavelet" has followed the opposite trajectory as "convolutional neural network"

A current trend of audio signal processing research is to apply 2D convolutional neural networks to time-frequency representations of audio. Although great progress was made in the pre-deep learning era on perfect reconstruction filterbanks and wavelet transforms, most of the algorithms and techniques have been abandoned by practitioners of deep learning because no standardized and free implementations were ever widely disseminated. As a result, the standarized set of triangular "Mel" filterbanks dating back nearly 50 years appears to have remained the dominant tool, and have even seen a resurgance despite their [impending obsolescence.][1]

![](img/trends2.png)

The use of these methods is accumulating a considerable debt. Compared to the most recent iterations of constant-Q filterbanks, these methods for time-frequency analysis lack several desirable properties

* Parameterization/tunability
* Perfect reconstruction
* Meaningful/interpretable phase representation
* Generalization to multiple channels/phased arrays
* Efficient implementation for oversampled representations.

[1]:https://asmp-eurasipjournals.springeropen.com/articles/10.1186/s13636-021-00217-4

# Filter Banks and Wavelet Transforms

Filter banks are the oldest method of time-frequency analysis, dating back to the first analog spectrum analyzer built by Hermann von Helmholtz in the 19th century.

![](img/helmholtz.png)

In 1909, mathematician Alfréd Haar discovered that a list of $2^n$ numbers can be represented by recursively taking the sum and difference of adjacent pairs. This property is now commonly referred to 'alias cancellation'

In the 1970s, engineers created the first discrete, invertible filterbanks by expoiting this property. Using what are known as 'conjugate mirror filters'. In the following decade, a massive research effort took place, resulting in a rigorous mathematical understanding of these processes and the development of various "wavelet transforms."

Unfortunately, deep learning has begun to displace these techniques in education, leading to considerable confusion and technical debt.

# Haar Decomposition
(Draft) Haar decomposition is one of the simplest wavelet transform that has perfect reconstruction property, thus is a good introduction example. First of all, note that the input is a sequence of integers, because we are dealing digital data after sampling from raw data, such as audio. By performing two convolutions with the input, we get a low-frequency component and a high-frequency component.
As we can see, the sum and difference of the high and low frequency components forms a splitted signal of the original signal, which we can merge to retain the original one. This is an example of “critically sampled” dyadic filterbank, which means that the size of data after applying the filter is exactly same as original data. Note that, “dyadic filters” refers “dynamic analysis filters”, the type of filtering that consists of a filtering followed by downsampling. There is an inverse operation, which is synthetic filters. (Which we are dealing by convolution. We will get to this detail later)

In [ ]:
x = round.(Int,10*randn(8));

In [ ]:
print(x)

[7, 1, -13, 20, 4, 7, -18, 5]

In [ ]:
s = conv(x,[1, 1])[2:2:end];
d = conv(x,[1,-1])[2:2:end];

In [ ]:
println("s = ",s); print("d = ",d)

s = [8, 7, 11, -13]
d = [-6, 33, 3, 23]

In [ ]:
y1 = (s-d)/2;
y2 = (s+d)/2;

In [ ]:
println(Int.(y1)); print(Int.(y2))

[7, -13, 4, -18]
[1, 20, 7, 5]

This is an example of a **critically sampled** dyadic filterbank

# Time-frequency tiling



Decomposing the lower branch produces octave filters

img/dyadic_analysis_filterbank.svg

![](img/octave_tiling.png)


Decomposing both branches (balanced tree)

(uniform grid of rectangles) 

you can have anything in the middle

(Draft)
In the Haar decomposition, we received two signals with half time resolution but twice the frequency resolution signals, by applying the filters. Next, we introduce a more fine-grained time-frequency tiling pattern, as shown in the image.

This structure has the benefit of a simple data structure, since each tile is a vector, while providing the exact trade-off between the frequency resolution and the time resolution as we want.

# Decimation filterbank

filter then downsample

!= dilation + stride (but close)

show toeplitz matrix multiplication with upsampling/downsampling

# Polyphase filterbank

upsample then filter
 
?= transposed convolution with dilation and stride

Another oppurtunity to add CPU parallelism

show mathematically that it's equivalent to splitting filter into L pieces where L is the upsampling factor, and applying filters in parallel



# Oversampled filterbanks

(Draft) Oversampled filterbanks refers to the type of filterbank that results in larger sized output data compared to the input. For example, Fourier Transform is an oversampled filtering with factor 2, as it yields a signal in frequency domain and the phase domain. 

Because of this nature, oversampled filterbank can get better CPU parallelism by having multiple data pipeline.

# Undersampled filterbanks

(Draft) In contrast to oversampled filterbanks, undersampled filterbank results in smaller sized output data compared to the input data. An example is convolving input signal with a small kernal without paddings.

# Analysis filterbank

# Synthesis filterbank

# Processing

* denoising
* compression
* source separation

# Octave Filters + Balanced Filters

In our filter, we repeatedly apply analysis filters to break down the input signal and synthesis filters to reconstruct from broken down (and possibly processed) pieces. As shown in the earlier image, this has an unbalanced tree structure where we apply analysis filters to the higher frequency-band output at each level, but leave the lower frequency-band output. This is a natural structure, as human ear processes sound signals in a similar way. 

In our algorithm, we break down the signal into 11 octaves, as 11 is mostly sufficient to cover human’s hearing capability. As a result, we get 11 vectors, where higher frequency blocks have better resolution.  Note that, we can apply any operation on these vectors and still retain the reconstruction property, as long as the operation has the mirror-conjugate property.

Following the octave-spaced analysis filter, we apply dyadic filters in a balanced-tree fashion (i.e. apply filters to both high and low frequency outputs), to obtain some fixed frequency-resolution blocks out of the imbalanced length blocks from the previous step. 

Now, we have obtained 11 blocks of signals, decomposed from the orignal signal, where each block has the exact same frequency resolutions. We can see this as a feature representatino of the original data, where the features size is exactly same as original (i.e. critially sampled). 

Later, we will demonstrate the ease of use with this decomposition. We can perform various useful operations such as denoising and comporession with very simple code, and it is highly efficient due to massive parallelism.

# Demo our program